In [4]:
import theano
import numpy as np
from theano import tensor as T

trX = np.linspace(-1, 1, 101)
trY = 2 * trX + np.random.randn(*trX.shape) * 0.2

x = T.scalar()
y = T.scalar()

def model(x, w):
    return x * w

w = theano.shared(0.0)
y_hat = model(x, w)

cost = T.sqr(y_hat - y)
gradient = T.grad(cost=cost, wrt=w)
updates = [[w, w - gradient * 0.0005]]

train = theano.function(inputs=[x, y], outputs=cost, updates=updates)

w_trace = []
for i in range(100):
    w_trace.append(w.get_value())
    for tr_x, tr_y in zip(trX, trY):
        train(tr_x, tr_y)
w_trace.append(w.get_value()) 
print w.get_value()

In [6]:
from IPython.display import HTML
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""


def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

In [7]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import animation


fig = plt.figure()
ax = plt.axes(xlim=(-1.5, 1.5), ylim=(-3, 3))
line, = ax.plot([], [], 'r', lw=2)
data, = ax.plot(trX, trY, 'bo')

def init():
    line.set_data([], [])
    return line, data

def animate(i):
    line.set_data(trX, w_trace[i] * trX)
    ax.set_title('current slope %0.2f' % w_trace[i])
    return line, ax

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=100)
display_animation(anim)